In [1]:
import os
import pickle

# input 
data_directory = ".//hooklogs//morstar/" # make sure the last character is '/'
in_tag = "morstar"
in_parseFirstPar = True # keep first parameter or not
in_window = 1

outputPath = ".//output//folderName/"

In [2]:
# 1.Get a dict of feature profiles (out: a hkName list, a featureProfile dict)
%run FeatureHooklog3.ipynb
Hooklog = FeatureHooklog3

hkName_li = list(filter(lambda f:f.endswith('.trace.hooklog'), os.listdir(data_directory))) # hooklog Name List

# ps: get Hkli without timestamp
fp_dict = {f:Hooklog(data_directory + f, 1).getHkli_noContainTS() for f in hkName_li } # Feature Profile dict (of hkName_li)
execTrace_dict = {f:Hooklog(data_directory + f, 1).getHkli_containTS() for f in hkName_li } # execution trace dict

# 2.Get a dict of pairwise alignment result 
# (in: a featureProfile dict; out: alignmentProfile dict, alingmnt baseline)

%run Alignment3.ipynb
BASE = hkName_li[0] # randomly pick a BASE trace log
fpBASE = fp_dict[BASE]
align_dict = {hk:pairwise_NW( fpBASE, fp_dict[hk], 2, -1, -3, 1, 0, 1)[2] for hk in hkName_li } # pairwise all

# 3. Get matchMatrix, gapSeqList
# (in: alignmentProfile dict; out: matchMatrix, gapSeqList)

%run StructMatchGap3.ipynb
data = structMatchGap(align_dict)
matchMatrix = data[0]
gapSeqList = data[1]

# 4. Get stageMatrix 
# (in: matchMatrix, gapSeqList; out: stageMatrix)

% run StageMatrix.ipynb
stageMatrixResult = stageMatrix(matchMatrix, gapSeqList)

#  5. Get Motif Elements
# (in: stageMatrixResult, BASE; out: Motif object)

% run Motif.ipynb
Motif = Motif(stageMatrixResult, BASE)

total segment: 10
No Gap.


In [3]:
######  Garbage Code  -->>>  Work around only!   #####
###  WJ:  I don't know why do this!?

with open('.//docs//label_dict-m2t-t2m-2014.pickle', 'rb') as f:
    label_dict = pickle.load(f)[1] #(proc_list,proc_dict,label_list, label_dict)
label_dict = {k[:-len('.trace.hooklog')]:label_dict[k] for k in label_dict}

In [4]:
## Drawing stages graphs
## output into 'outputPath'

% run OutputStageBarchart3.ipynb
outputStage = OutputStage( stageMatrixResult, outputPath, BASE, Motif ) # output stages graphs
OutputMotiGraph(stageMatrixResult, BASE, fp_dict, outputStage, outputPath, Motif, label_dict) # output motif graphs

==Starting drawing stages==
---Node Build Done---
0 stage
1 stage
2 stage
3 stage
4 stage
5 stage
6 stage
Warning!
	Does not run others, because only one stage is unnecessary if there is no gap!
---Node connect done---
---Set Label Done---
---Save Output Done---
==Starting drawing motifs==
---Save featureProfile Plot done---
---Save MotifLen Prob Plot done---
---Save Distinct motif Plot done---


In [5]:
## Output common motif analysis
# output into 'outputPath'

% run CommonMotifAnalysis3.ipynb
motiDict = MotiDict(stageMatrixResult, Motif, execTrace_dict, outputStage)
comMotif = motiDict.getComMoti()
comResAnalysis = ComResAnalysis(comMotif)

==Calculate common moitf==
--- 1 set CommontMotif dict fin---
--- 2 set DistinctMotif dict fin---
==Analyzing common motif resources==
---1 Set API Done---
CreateProcess
---2 Set SC_API res done---
	test for codeblock
---3 Check Motif subset---
---4 Set Res Statistics---
---5 Set Winnow res---


In [6]:
% run OutputCommonMotif3.ipynb
OutputCsv(outputPath, comResAnalysis, stageMatrixResult, Motif, BASE, execTrace_dict, outputStage)
OutputComMotiAnaly(comResAnalysis, outputPath)

==Output CSVs==
--- 1.1 output StageMotiCsv---
--- 1.2 output TotalResCsv---
--- 2.1 output CatGraph---
--- 2.2 output ComMotiDetailAPI---
--- 2.3 output ResAPIcatStatCsv---
